In [70]:
!pip install torchmetrics

In [71]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import math

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [72]:
FILL_IN = "FILL_IN"

In [73]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-11-29 02:57:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.2’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.03s   

2025-11-29 02:57:09 (5.59 MB/s) - ‘smsspamcollection.zip.2’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [74]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [75]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [76]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-11-29 02:57:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.3’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.03s   

2025-11-29 02:57:09 (6.22 MB/s) - ‘smsspamcollection.zip.3’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [77]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [78]:
text = df.text.values
labels = df.label.values

In [79]:
# Get the bert-base-uncased tokenizer and set lower case to True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [80]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════════╤═════════════╕
│ Tokens       │   Token IDs │
╞══════════════╪═════════════╡
│ you          │        2017 │
├──────────────┼─────────────┤
│ available    │        2800 │
├──────────────┼─────────────┤
│ now          │        2085 │
├──────────────┼─────────────┤
│ ?            │        1029 │
├──────────────┼─────────────┤
│ i            │        1045 │
├──────────────┼─────────────┤
│ '            │        1005 │
├──────────────┼─────────────┤
│ m            │        1049 │
├──────────────┼─────────────┤
│ like         │        2066 │
├──────────────┼─────────────┤
│ right        │        2157 │
├──────────────┼─────────────┤
│ around       │        2105 │
├──────────────┼─────────────┤
│ hillsborough │       29330 │
├──────────────┼─────────────┤
│ &            │        1004 │
├──────────────┼─────────────┤
│ amp          │       23713 │
├──────────────┼─────────────┤
│ ;            │        1025 │
├──────────────┼─────────────┤
│ &            │        1004 │
├───────

In [81]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  encoding = tokenizer.encode_plus(
      input_text,
      add_special_tokens = True,
      max_length = 32,
      padding = 'max_length',
      truncation = True,
      return_attention_mask = True,
      return_tensors = 'pt'
  )

  return encoding


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


In [82]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │         101 │                1 │
├──────────┼─────────────┼──────────────────┤
│ did      │        2106 │                1 │
├──────────┼─────────────┼──────────────────┤
│ i        │        1045 │                1 │
├──────────┼─────────────┼──────────────────┤
│ forget   │        5293 │                1 │
├──────────┼─────────────┼──────────────────┤
│ to       │        2000 │                1 │
├──────────┼─────────────┼──────────────────┤
│ tell     │        2425 │                1 │
├──────────┼─────────────┼──────────────────┤
│ you      │        2017 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ?        │        1029 │                1 │
├──────────┼─────────────┼──────────────────┤
│ i        │        1045 │                1 │
├──────────┼─────────────┼──────────────────┤
│ want     │        2215 │        

In [83]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    range(len(token_id)),
    test_size = val_ratio,
    stratify=labels,
    random_state = 42
)

# Train and validation sets
train_set = TensorDataset(
    token_id[train_idx],
    attention_masks[train_idx],
    labels[train_idx]
)

val_set = TensorDataset(
    token_id[val_idx],
    attention_masks[val_idx],
    labels[val_idx]
)

# Prepare DataLoader
train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True
)

validation_dataloader = DataLoader(
    val_set,
    batch_size = batch_size
)

print(f"{len(train_set)} {len(val_set)}")

4459 1115


Define the LoRA specific layers.

In [164]:
import torch.nn as nn
import torch.nn.functional as F

# Define a LoRA Layer which has A, B and alpha parameters
class LoRALayer(torch.nn.Module):
  def __init__(self, in_dim, out_dim, rank, alpha):
    super().__init__()
    # Initialize A to be a parameter matrix of dimension in_dim by rank
    self.A = nn.Parameter(torch.randn(in_dim, rank))
    # Initialize all the elements of A via kaiming_uniform with a equal to sqrt(5) ?
    nn.init.kaiming_uniform_(self.A, a=0)
    # Initialize B to be a zero parameter matrix of the appropriate dimensions
    self.B = nn.Parameter(torch.zeros(rank, out_dim))
    self.alpha = alpha

  def forward(self, x):
    # Pass x through the LoRA layer and return the new x
    x = self.alpha * torch.matmul(torch.matmul(x, self.A), self.B)
    return x

# Define a class LoRALinear which has a linear layer and a LoRA layer on top
class LoRALinear(torch.nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(linear.in_features, linear.out_features, rank, alpha)

  def forward(self, x):
    # Pass x through the linear layer and also the lora layer
    return self.linear(x) + self.lora(x)

def lora_linear_replace(model, rank, alpha):
  # Use model.named_children to go through all layers and if the layer is Linear replace that layer with LoRALinear
  for name, module in model.named_children():
    # If the module is linear, replace the module in the model with a LoRA layer
    if isinstance(module, nn.Linear):
      # Replace the Linear layer with LoRALinear
      setattr(model, name, LoRALinear(module, rank, alpha))
    else:
      # Alterntively, recursively apply the same function to child modules so that other Linear layers get replaced
      lora_linear_replace(module, rank, alpha)

### Load specific versions of the model

In [165]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_attentions=False, output_hidden_states=False)


# Turn off all gradients of the model to start
for param in model.parameters():
    param.requires_grad = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [166]:
for name, parameters in model.named_parameters():
  print(f" name {name} parameter {parameters.numel()}")

 name bert.embeddings.word_embeddings.weight parameter 23440896
 name bert.embeddings.position_embeddings.weight parameter 393216
 name bert.embeddings.token_type_embeddings.weight parameter 1536
 name bert.embeddings.LayerNorm.weight parameter 768
 name bert.embeddings.LayerNorm.bias parameter 768
 name bert.encoder.layer.0.attention.self.query.weight parameter 589824
 name bert.encoder.layer.0.attention.self.query.bias parameter 768
 name bert.encoder.layer.0.attention.self.key.weight parameter 589824
 name bert.encoder.layer.0.attention.self.key.bias parameter 768
 name bert.encoder.layer.0.attention.self.value.weight parameter 589824
 name bert.encoder.layer.0.attention.self.value.bias parameter 768
 name bert.encoder.layer.0.attention.output.dense.weight parameter 589824
 name bert.encoder.layer.0.attention.output.dense.bias parameter 768
 name bert.encoder.layer.0.attention.output.LayerNorm.weight parameter 768
 name bert.encoder.layer.0.attention.output.LayerNorm.bias parameter 

In [167]:
from os import truncate


# Set to True if LORA is used; if False, fine_tune flag will be used to decide if you fine tune the entire model or just parts
use_lora  = True
# If this is False, turn off gradients
fine_tune = True
# Set total_parameters to 0; this will count the number of parameters in each case
total_parameters = 0

if use_lora:
  # Use the lora_linear to attach a LoRA layer to each linear later of the original BERT model
  lora_linear_replace(model, 8, 16)
else:
  # If fine_tune is off, turn off gradients for all layers other than classifier
  if not fine_tune:
    # Turn off all gradients; count just the 'classifier' layer which is the only one that has gradients
    for name, param in model.named_parameters():
      if 'classifier' in name:
        param.requires_grad = True
  else:
    # Turn off all gradients except for classifier
    for name, param in model.named_parameters():
      param.requires_grad = True

# Get the total number of parameters with gradients
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

if use_lora:
  assert(total_parameters == 1345552)
else:
  if fine_tune:
    assert(total_parameters == 109483778)
  else:
    assert(total_parameters == 1538)




### Set the model to the right device

In [168]:
import platform

# Pick the system you have and select GPU if you can
if platform.system() == 'Darwin':
    device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
elif platform.system() == 'Linux':
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
else:
    device = torch.device('cpu')
print(device)

cuda


In [169]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [170]:
# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

### Train the model

In [171]:
import torchmetrics
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = torchmetrics.Accuracy(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
precision = torchmetrics.Precision(task='binary').to(device)
auroc = torchmetrics.AUROC(task='binary').to(device)

In [172]:
model.device

device(type='cuda', index=0)

In [180]:
# Main training / validation loop
import tqdm
for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # batch = FILL_IN
        # Unpack the batch
        # print(f"{batch[0]} \n {batch[1]} \n {batch[2]} \n")
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Set the gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Backward pass
        loss = train_output.loss
        loss.backward()

        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        # batch = FILL_IN
        input_ids, input_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        with torch.no_grad():
          # Forward pass
            eval_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Calculate validation metrics
        # labels = FILL_IN
        predicted_labels = torch.argmax(eval_output.logits, dim=1)
        print(f"predicted_labels {predicted_labels} labels {labels}")

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(predicted_labels, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

  0%|          | 0/2 [00:13<?, ?it/s]


RuntimeError: The expanded size of the tensor (16) must match the existing size (11) at non-singleton dimension 0.  Target sizes: [16, 12, 32, 32].  Tensor sizes: [11, 1, 32, 32]

### Test on a specific sentence, see the outcome

In [174]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Ham


### Questions

Question 1: Run the above by fine tuning bert and the classfier head and by not doing this (using BERT as a feature encoder). What is the gap between this?

FILL_IN

